# Instalamos SPADE 

In [ ]:
pip install spade

# Primer ejemplo: Hola Mundo
Se crea un agente que simplemente imprime "Hola Mundo"

In [ ]:

from spade.agent import Agent
from spade.behaviour import OneShotBehaviour

class SimpleAgent(Agent):
    class InformBehaviour(OneShotBehaviour):
        async def run(self):
            print("Hola, soy un agente simple!")

    async def setup(self):
        self.add_behaviour(self.InformBehaviour())



# Importante: cambiar el nombre del agente si usamos el mismo servidor XMPP
Por ejemplo, usar el **login** personal como parte del nombre del agente

In [ ]:
agent = SimpleAgent("ag1_login@gtirouter.dsic.upv.es", "secret")
await agent.start()

# Segundo ejemplo: Crear un agente con un comportamiento
El agente imprime un valor y finaliza despues de 10 iteraciones

In [ ]:
import asyncio
from spade.agent import Agent
from spade.behaviour import CyclicBehaviour

class DummyAgent(Agent):
    class MyBehav(CyclicBehaviour):
        async def on_start(self):
            print("Starting behaviour . . .")
            self.counter = 0

        async def run(self):
            print("Counter: {}".format(self.counter))
            self.counter += 1

            if self.counter >= 10:
                self.kill()  # Finaliza el comportamiento después de 10 iteraciones
            else:
                await asyncio.sleep(1)

    async def setup(self):
        print("Agent starting . . .")
        b = self.MyBehav()
        self.add_behaviour(b)



In [ ]:
dummy = DummyAgent("ag1_login@gtirouter.dsic.upv.es", "secret") 
await dummy.start() 
print("DummyAgent started. Check its console to see the output.")

print("Wait until user interrupts with ctrl+C") 
await wait_until_finished(dummy) 


# Tercer ejemplo: Sender & Receiver


In [ ]:

from spade.agent import Agent
from spade.behaviour import OneShotBehaviour
from spade.message import Message

class SenderAgent(Agent):
    class SendBehaviour(OneShotBehaviour):
        async def run(self):
            msg = Message(to="ag2_login@gtirouter.dsic.upv.es")  # Destinatario
            msg.set_metadata("performative", "inform")  # Tipo de mensaje
            msg.body = "Hola desde el agente emisor!"

            await self.send(msg)
            print("Mensaje enviado!")

    async def setup(self):
        self.add_behaviour(self.SendBehaviour())

class ReceiverAgent(Agent):
    class ReceiveBehaviour(OneShotBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)  # Espera por un mensaje por 10 segundos
            if msg:
                print("Mensaje recibido:", msg.body)
            else:
                print("No se recibió ningún mensaje.")

    async def setup(self):
        self.add_behaviour(self.ReceiveBehaviour())



In [ ]:
sender = SenderAgent("ag1_login@gtirouter.dsic.upv.es", "secret")
receiver = ReceiverAgent("ag2_login@gtirouter.dsic.upv.es", "secret")
await receiver.start()
await sender.start()


# Código para generar un ejemplo de sistema multiagente con SPADE donde un agente envía mensajes de forma periódica y otro los recibe e imprime


In [ ]:

# Código para el sistema con dos agentes: uno envía mensajes periódicamente y el otro los recibe

from spade.agent import Agent
from spade.behaviour import CyclicBehaviour, PeriodicBehaviour
from spade.message import Message
import asyncio

class PeriodicSenderAgent(Agent):
    class SendPeriodicBehaviour(PeriodicBehaviour):
        async def run(self):
            msg = Message(to="ag2_login@gtirouter.dsic.upv.es")  # Asegúrate de reemplazar esto con el JID correcto del receptor
            msg.set_metadata("performative", "inform")
            msg.body = "Mensaje periódico del agente emisor"

            await self.send(msg)
            print("Mensaje enviado.")

    async def setup(self):
        # Ejemplo: enviar un mensaje cada 5 segundos
        periodic_behaviour = self.SendPeriodicBehaviour(period=5)
        self.add_behaviour(periodic_behaviour)

class ReceiverAgent(Agent):
    class ReceiveBehaviour(CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)  # Espera por un mensaje por 10 segundos
            if msg:
                print("Mensaje recibido:", msg.body)
            else:
                print("Tiempo de espera excedido sin recibir mensaje.")

    async def setup(self):
        self.add_behaviour(self.ReceiveBehaviour())





In [ ]:
# Ejemplo de cómo iniciar y detener los agentes
sender = PeriodicSenderAgent("ag1_login@gtirouter.dsic.upv.es", "secret")
receiver = ReceiverAgent("ag2_login@gtirouter.dsic.upv.es", "secret")
await receiver.start()
await sender.start()



# Ejemplo donde un agente propone a otros agentes que le envien un dato y simplemente lo almacena

In [ ]:
import spade
import asyncio
import random

class AgentePropositor(spade.agent.Agent):
    async def setup(self):
        print(f"{self.name} iniciado")
        self.add_behaviour(self.ProposeBehaviour())

    class ProposeBehaviour(spade.behaviour.OneShotBehaviour):
        async def run(self):
            receivers = ["agente_receptor1@gtirouter.dsic.upv.es", "agente_receptor2@gtirouter.dsic.upv.es", "agente_receptor3@gtirouter.dsic.upv.es"]
            for receiver in receivers:
                msg = spade.message.Message(
                    to=receiver,
                    body="propuesta",
                    metadata={"performative": "propose"}
                )
                await self.send(msg)
                print(f"Propuesta enviada a {receiver}")

            # Esperar y recoger respuestas
            self.agent.responses = []
            for _ in range(3):
                response = await self.receive(timeout=10)
                if response:
                    self.agent.responses.append(response.body)
            print("Respuestas recibidas:", self.agent.responses)

class AgenteReceptor(spade.agent.Agent):
    async def setup(self):
        print(f"{self.name} iniciado")
        self.add_behaviour(self.ReceiveBehaviour())

    class ReceiveBehaviour(spade.behaviour.CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)
            if msg:
                number = str(random.randint(1, 100))
                reply = msg.make_reply()
                reply.body = number
                await self.send(reply)
                print(f"{self.agent.name} respondió con {number}")



In [ ]:
# Crear agentes
agente_propositor = AgentePropositor("agente_propositor@gtirouter.dsic.upv.es", "secret")
agente_receptor1 = AgenteReceptor("agente_receptor1@gtirouter.dsic.upv.es", "secret")
agente_receptor2 = AgenteReceptor("agente_receptor2@gtirouter.dsic.upv.es", "secret")
agente_receptor3 = AgenteReceptor("agente_receptor3@gtirouter.dsic.upv.es", "secret")

# Iniciar agentes

await agente_receptor1.start()
await agente_receptor2.start()
await agente_receptor3.start()

await agente_propositor.start()

# Ejemplo de sistema donde un agente proponente envia una propuesta a otros agentes, recibe respuestas, las procesa y selecciona una contestando al elegido que ha sido seleccionado

In [ ]:
import spade
import asyncio
import random

class AgentePropositor(spade.agent.Agent):
    async def setup(self):
        print(f"{self.name} iniciado")
        self.add_behaviour(self.ProposeBehaviour())

    class ProposeBehaviour(spade.behaviour.OneShotBehaviour):
        async def run(self):
            receivers = ["agente_receptor1@gtirouter.dsic.upv.es", "agente_receptor2@gtirouter.dsic.upv.es", "agente_receptor3@gtirouter.dsic.upv.es"]
            self.agent.responses = {}

            # Enviar propuestas a cada receptor
            for receiver in receivers:
                msg = spade.message.Message(
                    to=receiver,
                    body="propuesta",
                    metadata={"performative": "propose"}
                )
                await self.send(msg)
                print(f"Propuesta enviada a {receiver}")

            # Esperar y recoger respuestas
            for _ in range(3):
                response = await self.receive(timeout=10)
                if response:
                    self.agent.responses[response.sender.localpart] = response.body

            print("Respuestas recibidas:", self.agent.responses)

            # Seleccionar y enviar respuesta
            if self.agent.responses:
                selected_agent, selected_value = random.choice(list(self.agent.responses.items()))
                response_msg = spade.message.Message(
                    to=f"{selected_agent}@localhost",
                    body=f"Tu valor {selected_value} fue seleccionado",
                    metadata={"performative": "inform"}
                )
                await self.send(response_msg)
                print(f"Enviada respuesta a {selected_agent} sobre valor seleccionado: {selected_value}")

class AgenteReceptor(spade.agent.Agent):
    async def setup(self):
        print(f"{self.name} iniciado")
        self.add_behaviour(self.ReceiveBehaviour())
        self.add_behaviour(self.InformBehaviour())

    class ReceiveBehaviour(spade.behaviour.CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)
            if msg and msg.metadata["performative"] == "propose":
                number = str(random.randint(1, 100))
                reply = msg.make_reply()
                reply.body = number
                await self.send(reply)
                print(f"{self.agent.name} respondió con {number}")

    class InformBehaviour(spade.behaviour.CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)
            if msg and msg.metadata["performative"] == "inform":
                print(f"{self.agent.name} recibió mensaje de selección: {msg.body}")





In [ ]:
# Crear agentes
agente_propositor = AgentePropositor("agente_propositor@gtirouter.dsic.upv.es", "secret")
agente_receptor1 = AgenteReceptor("agente_receptor1@gtirouter.dsic.upv.es", "secret")
agente_receptor2 = AgenteReceptor("agente_receptor2@gtirouter.dsic.upv.es", "secret")
agente_receptor3 = AgenteReceptor("agente_receptor3@gtirouter.dsic.upv.es", "secret")

# Iniciar agentes

await agente_receptor1.start()
await agente_receptor2.start()
await agente_receptor3.start()
await asyncio.sleep(1)
await agente_propositor.start()

# Ejemplo generalizado para un proponente y n agentes receptores

In [ ]:
import spade
import asyncio
import random

class AgentePropositor(spade.agent.Agent):
    async def setup(self):
        print(f"{self.name} iniciado")
        self.add_behaviour(self.ProposeBehaviour())

    class ProposeBehaviour(spade.behaviour.OneShotBehaviour):
        async def run(self):
            # Enviar propuestas a cada receptor
            for i in range(self.agent.num_receptores):
                receiver = f"agente_receptor{i}@gtirouter.dsic.upv.es"
                msg = spade.message.Message(
                    to=receiver,
                    body="propuesta",
                    metadata={"performative": "propose"}
                )
                await self.send(msg)
                print(f"Propuesta enviada a {receiver}")

            # Esperar y recoger respuestas
            for _ in range(self.agent.num_receptores):
                response = await self.receive(timeout=10)
                if response:
                    print(f"Respuesta recibida de {response.sender.localpart}: {response.body}")

class AgenteReceptor(spade.agent.Agent):
    async def setup(self):
        print(f"{self.name} iniciado")
        self.add_behaviour(self.ReceiveBehaviour())

    class ReceiveBehaviour(spade.behaviour.CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)
            if msg:
                number = str(random.randint(1, 100))
                reply = msg.make_reply()
                reply.body = number
                await self.send(reply)
                print(f"{self.agent.name} respondió con {number}")

async def crear_agentes_receptores(num_receptores):
    receptores = []
    for i in range(num_receptores):
        agente = AgenteReceptor(f"agente_receptor{i}@gtirouter.dsic.upv.es", "secret")
        await agente.start()
        receptores.append(agente)
    return receptores




In [ ]:
num_receptores = 5

# Crear y empezar agentes receptores
receptores = await crear_agentes_receptores(num_receptores)

# Crear y empezar agente propositor
agente_propositor = AgentePropositor("agente_propositor@gtirouter.dsic.upv.es", "secret")
agente_propositor.num_receptores = num_receptores
await agente_propositor.start()